# Connection Viya and Python

In [1]:
# library import
import os
import numpy as np
import scipy as sp
import pandas as pd
import tensorflow as tf

%matplotlib inline
import matplotlib.pyplot as plt


In [2]:
import swat

os.environ['CAS_CLIENT_SSL_CA_LIST'] = r'C:\Users\korbsl\OneDrive - SAS\Backup\800_Install\20_Python\trustedcerts.pem'

hostname = 'casc.kolon.com'
port = 8777
username = 'viya_sask01'
password = 'viya'

# Connect to CAS
conn = swat.CAS(hostname = hostname,
                port = port,
                username = username,
                password = password,
                protocol = 'https')
# serverStatus action in builtins action set
print(conn.builtins.serverStatus())
print(conn)


SWATError: Unable to connect to any URL: https://casc.kolon.com:8777

In [ ]:
#conn.terminate() 

CIFAR10 data import

In [8]:
from tensorflow.keras.datasets.cifar10 import load_data

In [31]:
def next_batch(num, data, labels):
    ''' num 개수 만큼의 랜덤한 샘플들과 레이블들을 리턴 '''
    idx = np.arange(0, len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]
    
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)


In [55]:
def build_CNN_classifier(x):
    x_image = x
    
    W_conv1 = tf.Variable(tf.truncated_normal(shape=[5,5,3,32], stddev=5e-2))
    b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))
    h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1], padding='SAME') + b_conv1)
    
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')
    
    # 두번째 convolutional layer - 32개의 특징들(feature)을 64개의 특징들(feature)로 맵핑(maping)합니다.
    W_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 32, 64], stddev=5e-2))
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)

    # 두번째 pooling layer.
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

    # 세번째 convolutional layer
    W_conv3 = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], stddev=5e-2))
    b_conv3 = tf.Variable(tf.constant(0.1, shape=[128]))
    h_conv3 = tf.nn.relu(tf.nn.conv2d(h_pool2, W_conv3, strides=[1, 1, 1, 1], padding='SAME') + b_conv3)

    # 네번째 convolutional layer
    W_conv4 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
    b_conv4 = tf.Variable(tf.constant(0.1, shape=[128])) 
    h_conv4 = tf.nn.relu(tf.nn.conv2d(h_conv3, W_conv4, strides=[1, 1, 1, 1], padding='SAME') + b_conv4)

    # 다섯번째 convolutional layer
    W_conv5 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
    b_conv5 = tf.Variable(tf.constant(0.1, shape=[128]))
    h_conv5 = tf.nn.relu(tf.nn.conv2d(h_conv4, W_conv5, strides=[1, 1, 1, 1], padding='SAME') + b_conv5)
    
    W_fc1 = tf.Variable(tf.truncated_normal(shape=[8*8*128, 384], stddev=5e-2))
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[384]))
    
    h_conv5_flat = tf.reshape(h_conv5, [-1, 8*8*128])
    h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1)+b_fc1)
    
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    W_fc2 = tf.Variable(tf.truncated_normal(shape=[384,10], stddev=5e-2))
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
    logits = tf.matmul(h_fc1_drop, W_fc2)+b_fc2
    y_pred = tf.nn.softmax(logits)
    
    return y_pred, logits

In [56]:
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

In [57]:
(x_train, y_train), (x_test, y_test) = load_data()

In [58]:
y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10),axis=1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test, 10),axis=1)

In [59]:
y_pred, logits = build_CNN_classifier(x)

In [60]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
train_step = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

In [61]:
correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [62]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(100):
        batch = next_batch(128, x_train, y_train_one_hot.eval())
        
        if i % 10 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y:batch[1], keep_prob: 1.0})
            loss_print = loss.eval(feed_dict={x:batch[0], y:batch[1], keep_prob: 1.0})
            
            print("반복(Epoch): %d, 트레이닝 데이터 정확도 : %f, 손실함수(loss) : %f" % (i,train_accuracy, loss_print))
        sess.run(train_step, feed_dict={x:batch[0], y:batch[1], keep_prob:0.8})
        
    test_accuracy = 0.0
    
    for i in range(10):
        test_batch = next_batch(1000, x_test, y_test_one_hot.eval())
        test_accuracy = test_accuracy + accuracy.eval(feed_dict={x:batch[0], y:batch[1], keep_prob:1.0})
        
    test_accuracy = test_accuracy/10
    print("테스트 데이터 정확도: %f" % test_accuracy)

반복(Epoch): 0, 트레이닝 데이터 정확도 : 0.132812, 손실함수(loss) : 186.937134
반복(Epoch): 10, 트레이닝 데이터 정확도 : 0.210938, 손실함수(loss) : 2.992003
반복(Epoch): 20, 트레이닝 데이터 정확도 : 0.125000, 손실함수(loss) : 2.940197
반복(Epoch): 30, 트레이닝 데이터 정확도 : 0.187500, 손실함수(loss) : 2.378651
반복(Epoch): 40, 트레이닝 데이터 정확도 : 0.117188, 손실함수(loss) : 2.748670
반복(Epoch): 50, 트레이닝 데이터 정확도 : 0.265625, 손실함수(loss) : 2.121551
반복(Epoch): 60, 트레이닝 데이터 정확도 : 0.195312, 손실함수(loss) : 2.172610
반복(Epoch): 70, 트레이닝 데이터 정확도 : 0.164062, 손실함수(loss) : 2.309586
반복(Epoch): 80, 트레이닝 데이터 정확도 : 0.148438, 손실함수(loss) : 2.339225
반복(Epoch): 90, 트레이닝 데이터 정확도 : 0.171875, 손실함수(loss) : 2.205327
테스트 데이터 정확도: 0.148438
